## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка

**Цель исследования**
    
Выявить, существует ли зависимость между семейным положением и количеством детей на погашение кредита в срок.

**План работы**
1. Выгрузить датафрейм с данными;
2. Посмотреть общую информацию о данных;
3. Избавиться от дубликатов и пропусков, если таковы имеются;
4. Проанализировать данные;
5. Сделать соответствующие выводы.
    
**Входные данные от банка — статистика о платёжеспособности клиентов.**

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [ ]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [ ]:
data.head(20)

In [ ]:
data.info()

**Вывод**

В 2-х столбцах датафрейма есть одинаковое количество пропусков. Также в столбце "days_employed" заметили отрицательные значения, которых не должно быть, поскольку этот столбец означает количество рабочих дней.

### Шаг 2. Предобработка данных

### Удаление пропусков

In [ ]:
data.isna().sum()

In [ ]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

In [ ]:
data['days_employed'] = data['days_employed'].abs()

In [ ]:
data.groupby('income_type')['days_employed'].agg('median')

In [ ]:
data['children'].unique()

In [ ]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [ ]:
data['children'].unique()

### Удаление пропусков

In [ ]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = data.loc[(data['income_type'] == t), 'days_employed'].median()

In [ ]:
data.isna().sum()

### Изменение типов данных

In [ ]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

In [ ]:
data['education'] = data['education'].str.lower()

In [ ]:
data.duplicated().sum()

In [ ]:
data = data.drop_duplicates()

### Категоризация данных

In [ ]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [ ]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [ ]:
data['purpose'].unique()

In [ ]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [ ]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

**Вывод**

Мы предобработали данные: заменили пропуски в столбцах, удалили аномальные значения, привели данные к общему виду и удалили дубликаты.
Произвели категоризацию для удобства анализа данных и вывели результат в новые столбцы.

### Шаг 3. Ответим на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
data.pivot_table(index='children', values='debt', aggfunc={'count', 'mean'})

**Вывод**

Самый большой процент не возвратов денег в срок прослеживается среди людей с 4-я детьми. Это скорее всего связано с маленьким числом заёмщиков в этой категории.

Можно сказать, что чаще всего не выплачивают кредит в срок категории с 2 и 3 детьми, потому что в этих случаях число заёмщиков достаточно значительное.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
data.pivot_table(index='family_status', values='debt', aggfunc={'count', 'mean'})

**Вывод**

Исходя из данных сводной таблицы можно сделать вывод, что не женатые и находящиеся в гражданском браке люди чаще не выплачивают кредиты в срок.

В большей степени просрочку допускают заемщики, не обремененные семейными узами - 9,75 % от общего количества всех не женатых / не замужних, но также недалеко отстают от них лица, живущие в гражданском браке - 9,35 % от общего числа состоящих в гражданском браке.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
import numpy as np
total_income_debt=pd.DataFrame(df, columns=['total_income', 'debt'])
total_income_debt['income_groups'] = pd.cut(total_income_debt['total_income'],[0,50000,100000,150000,200000,np.inf],labels=["до 50 тыс. руб.", "50-100 тыс. руб.", "100-150 тыс. руб.","150-200 тыс. руб.","свыше 200 тыс. руб."])
total_income_debt=total_income_debt.groupby('income_groups')['debt'].agg(['count','sum'])
total_income_debt['%']=round((total_income_debt[('sum')]/total_income_debt[('count')])*100,2)
total_income_debt.columns = ['Общее количество заемщиков','Количество просрочников','%']
total_income_debt

,Общее количество заемщиков,Количество просрочников,%
income_groups,,,
до 50 тыс. руб.,372,23,6.18
50-100 тыс. руб.,4091,331,8.09
100-150 тыс. руб.,7160,624,8.72
150-200 тыс. руб.,4764,405,8.50
свыше 200 тыс. руб.,5067,358,7.07


- Как разные цели кредита влияют на его возврат в срок?

In [ ]:
data.pivot_table(index='purpose_category', values='debt', aggfunc={'count', 'mean'})

**Вывод**

Заемщики, которые брали кредиты для операций с автомобилем имеют 9,3% не выплат в срок от общего числа. Для образования такой процент составляет 9,2.

 - Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
data.pivot_table(index='total_income_category', values='debt', aggfunc={'count', 'mean'})

**Вывод**

Заемщики с самым низким доходом чаще не выплачивают кредиты в срок. На это повлияло то, что количество заемщиков в этой категории меньше всего. Затем следуют люди, получающие от 50 до 200 тысяч, их процент не выплат от общего числа составляет 8,4.

### Шаг 4. Общий вывод

За основу анализа данных была взята статистика о платежеспособности клиентов банка. 

На этапе преобработки провели следующие операции с данными:
 - Вывели общую информацию о датафрейме
 - Заполнили пропуски, удалили аномальные значения и дубликаты
 - Привели данные к общему виду
 - Произвели категоризацию по количеству дохода и цели кредита

Отвечая на вопрос, есть ли зависимость между семейным положением, количеством детей и выплатой кредита в срок, выяснили, что при наличии 2-х детей люди чаще просрочивают оплату кредита, также не выплачивают в срок не женатые и состоящие в гражданском браке, имеющие доход от 50 до 200 тысяч и те, кто берёт кредит для операций с автомобилем.

Идеальным заёмщиком будет человек без детей, женатый, получающий от 200 тысяч до 1 миллиона и целью кредита у которого будет проведение свадьбы или операции с недвижимостью.